# Project Time domain data into lower dimensions
In this section, we will use UMAP (Uniform Manifold Approximation and Projection) to reduce the dimensionality of our time domain data from 1024 points down to just 2. This reduction allows us to visualize the structure of our high dimensional data in a 2D plot.

But why would we want to do this?

Even though each sample contains 1024 time domain values, it’s difficult to build an intuition about the data or spot patterns when we’re working in such high dimensional space. By projecting the data into two dimensions while preserving as much of its underlying structure as possible, we can:
	•	Explore the data visually: Clusters in the 2D plot might correspond to different physical properties of the samples (e.g., number of layers in a material, material type, etc.).
	•	Gain insights into separability: If we see well-defined clusters, that suggests the data contains meaningful differences that could be exploited by a machine learning model.
	•	Detect outliers: Points that lie far from any cluster may represent unusual or erroneous data.

UMAP is particularly well suited for this kind of task because it preserves both the local and global structure of the data better than older methods like PCA or t-SNE, especially when the data lies on a nonlinear manifold.


NOTE: The supporting data is not uploaded in this repository. Similar datasets can be generated with Data_gen_efficient.py

In [1]:
import torch
import umap
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler

In [2]:
# Load the dataset with weights_only=False
data = torch.load("INSERT-PATH-TO-DATA", weights_only=False)

# Extract components
synthetic_data = data["synthetic_data"]
material_params = data["material_params"]
num_layers = data["num_layers"]

print("Dataset loaded successfully!")
print(f"Number of samples: {len(synthetic_data)}")
print(f"Shape of synthetic_data: {synthetic_data.shape}")
print(f"Shape of num_layers: {num_layers.shape}")
print(f"Example number of layers: {num_layers[:10]}")

num_classes = int(max(num_layers)) # class labels start at 1 not 0.

FileNotFoundError: [Errno 2] No such file or directory: 'INSERT-PATH-TO-DATA'

In [ ]:
# Convert to numpy for UMAP
X = synthetic_data.numpy()
y = num_layers.numpy()

# Run UMAP
reducer = umap.UMAP(n_components=2, random_state=42)
X_umap = reducer.fit_transform(X)

In [ ]:
# Plot UMAP projection
plt.figure(figsize=(9, 8))
palette = sns.color_palette("deep", num_classes)

# Labels start at 1, adjust for color mapping
sns.scatterplot(x=X_umap[:, 0], y=X_umap[:, 1], hue=y, palette=palette, legend='full', s=10)
plt.title("UMAP Projection of Synthetic Time-Domain Data")
plt.xlabel("UMAP Dimension 1")
plt.ylabel("UMAP Dimension 2")
plt.legend(title="Number of Layers", loc="best")
plt.tight_layout()
plt.show()

In [ ]:
# Plotting
fig, axs = plt.subplots(1, 2, figsize=(14, 6))

# Left: n (real part of first layer)
sc1 = axs[0].scatter(X_umap[:, 0], X_umap[:, 1], c=n_real, cmap='viridis', s=5)
axs[0].set_title("UMAP Projection Colored by Refractive Index (n₁)")
axs[0].set_xlabel("UMAP Dimension 1")
axs[0].set_ylabel("UMAP Dimension 2")
plt.colorbar(sc1, ax=axs[0], label="Re(n₁)")

# Right: total thickness
sc2 = axs[1].scatter(X_umap[:, 0], X_umap[:, 1], c=total_thickness, cmap='plasma', s=5)
axs[1].set_title("UMAP Projection Colored by Total Thickness")
axs[1].set_xlabel("UMAP Dimension 1")
axs[1].set_ylabel("UMAP Dimension 2")
plt.colorbar(sc2, ax=axs[1], label="Thickness (m)")

plt.tight_layout()
plt.show()

In [ ]:
# Step 1: Extract total thickness per sample
total_thicknesses = []
for sample in material_params:
    total_thickness = sum(D.real for (_, D) in sample)
    total_thicknesses.append(total_thickness)

total_thicknesses = np.array(total_thicknesses).reshape(-1, 1)  # shape: (N, 1)

# Step 2: Prepare the pulse data
X_pulses = synthetic_data.numpy()  # shape: (N, 1024)
y = num_layers.numpy()             # class labels (1 to 5)

# Step 3: Concatenate pulses with total thickness
# Normalize both parts first
scaler_pulse = StandardScaler()
scaler_thick = StandardScaler()

X_pulses_scaled = scaler_pulse.fit_transform(X_pulses)
thickness_scaled = scaler_thick.fit_transform(total_thicknesses)

X_augmented = np.hstack([X_pulses_scaled, thickness_scaled])  # shape: (N, 1025)

# Step 4: Run UMAP
reducer = umap.UMAP(n_components=2, random_state=42)
X_umap = reducer.fit_transform(X_augmented)

# Step 5: Plot
plt.figure(figsize=(9, 8))
palette = sns.color_palette("deep", np.max(y))  # handles labels 1 to 5

sns.scatterplot(x=X_umap[:, 0], y=X_umap[:, 1], hue=y, palette=palette, legend='full', s=10)
plt.title("UMAP Projection of Time-Domain Pulses + Total Thickness")
plt.xlabel("UMAP Dimension 1")
plt.ylabel("UMAP Dimension 2")
plt.legend(title="Number of Layers", loc="best")
plt.tight_layout()
plt.show()